In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve,confusion_matrix,f1_score, roc_auc_score
from keras.layers import Concatenate, Dot, Input, LSTM
from keras.layers import RepeatVector, Dense, Activation
from keras.layers import Reshape, Dropout, Add, Subtract, Flatten, Embedding
#from keras.optimizers import Adam

from keras.models import load_model, Model
import keras.backend as K
import warnings
warnings.filterwarnings("ignore")

from process_data import *

In [2]:
most_fre('north')

'n'

In [12]:
data = pd.read_excel("fake_data.xlsx")

In [13]:
cat(data, "path", "leng_path", s = ">")
cat(data,'marketing_area',"leng_area",s=',')
cat(data,'tier',"leng_tier",s=',')
cat(data,'customer_type',"leng_type",s=',')

In [15]:
data.head()

,path_id,path,total_conversions,last_time_lapse,null_conversion,marketing_area,tier,customer_type,leng_path,leng_area,leng_tier,leng_type
0,fake001,"[A1, A2, A3, A4]",1,"588241,532798,520504,515807,0",unknown,[north],[t1],[c1],4,1,1,1
1,fake002,"[A1, A2, A3, A4]",0,"18337,178,0",unknown,[south],[t2],[c1],4,1,1,1
2,fake003,"[A1, A2, A3]",1,"18337,178,0",unknown,[west],[t3],[c1],3,1,1,1
3,fake004,"[A1, A2, A3, A4]",0,"588241,532798,520504,515807,0",unknown,[east],[t3],[c2],4,1,1,1
4,fake005,"[A1, A2, A3, A4]",1,"18337,178,0",unknown,[west],[t3],[c2],4,1,1,1


In [16]:
data_new = data[data.leng_path >= 3]
data_new.head()

,path_id,path,total_conversions,last_time_lapse,null_conversion,marketing_area,tier,customer_type,leng_path,leng_area,leng_tier,leng_type
0,fake001,"[A1, A2, A3, A4]",1,"588241,532798,520504,515807,0",unknown,[north],[t1],[c1],4,1,1,1
1,fake002,"[A1, A2, A3, A4]",0,"18337,178,0",unknown,[south],[t2],[c1],4,1,1,1
2,fake003,"[A1, A2, A3]",1,"18337,178,0",unknown,[west],[t3],[c1],3,1,1,1
3,fake004,"[A1, A2, A3, A4]",0,"588241,532798,520504,515807,0",unknown,[east],[t3],[c2],4,1,1,1
4,fake005,"[A1, A2, A3, A4]",1,"18337,178,0",unknown,[west],[t3],[c2],4,1,1,1


In [17]:
data_new = data_new.reset_index()
data_new.head()

,index,path_id,path,total_conversions,last_time_lapse,null_conversion,marketing_area,tier,customer_type,leng_path,leng_area,leng_tier,leng_type
0,0,fake001,"[A1, A2, A3, A4]",1,"588241,532798,520504,515807,0",unknown,[north],[t1],[c1],4,1,1,1
1,1,fake002,"[A1, A2, A3, A4]",0,"18337,178,0",unknown,[south],[t2],[c1],4,1,1,1
2,2,fake003,"[A1, A2, A3]",1,"18337,178,0",unknown,[west],[t3],[c1],3,1,1,1
3,3,fake004,"[A1, A2, A3, A4]",0,"588241,532798,520504,515807,0",unknown,[east],[t3],[c2],4,1,1,1
4,4,fake005,"[A1, A2, A3, A4]",1,"18337,178,0",unknown,[west],[t3],[c2],4,1,1,1


In [19]:
data_new.marketing_area = data_new.marketing_area.map(lambda x:most_fre(x))
data_new.head()

,index,path_id,path,total_conversions,last_time_lapse,null_conversion,marketing_area,tier,customer_type,leng_path,leng_area,leng_tier,leng_type
0,0,fake001,"[A1, A2, A3, A4]",1,"588241,532798,520504,515807,0",unknown,north,[t1],[c1],4,1,1,1
1,1,fake002,"[A1, A2, A3, A4]",0,"18337,178,0",unknown,south,[t2],[c1],4,1,1,1
2,2,fake003,"[A1, A2, A3]",1,"18337,178,0",unknown,west,[t3],[c1],3,1,1,1
3,3,fake004,"[A1, A2, A3, A4]",0,"588241,532798,520504,515807,0",unknown,east,[t3],[c2],4,1,1,1
4,4,fake005,"[A1, A2, A3, A4]",1,"18337,178,0",unknown,west,[t3],[c2],4,1,1,1


In [20]:
data_new.tier = data_new.tier.map(lambda x:most_fre(x))
data_new.customer_type = data_new.customer_type.map(lambda x:most_fre(x))
data_new.replace('','NA',inplace=True)

In [21]:
y = data_new.total_conversions

In [22]:
y

0     1
1     0
2     1
3     0
4     1
5     1
6     0
7     1
8     0
9     1
10    1
11    1
12    0
13    1
14    1
15    0
16    1
Name: total_conversions, dtype: int64

In [23]:
idx = [x for x in range(data_new.shape[0])]
np.random.seed(111)
random.shuffle(idx)
tr_idx = idx[0:int(0.9*len(idx))]
te_idx = idx[int(0.9*len(idx)):]

In [24]:
cat(data_new,'last_time_lapse',"leng_time_lapse",s=',')
data_new.head()

,index,path_id,path,total_conversions,last_time_lapse,null_conversion,marketing_area,tier,customer_type,leng_path,leng_area,leng_tier,leng_type,leng_time_lapse
0,0,fake001,"[A1, A2, A3, A4]",1,"[588241, 532798, 520504, 515807, 0]",unknown,north,t1,c1,4,1,1,1,5
1,1,fake002,"[A1, A2, A3, A4]",0,"[18337, 178, 0]",unknown,south,t2,c1,4,1,1,1,3
2,2,fake003,"[A1, A2, A3]",1,"[18337, 178, 0]",unknown,west,t3,c1,3,1,1,1,3
3,3,fake004,"[A1, A2, A3, A4]",0,"[588241, 532798, 520504, 515807, 0]",unknown,east,t3,c2,4,1,1,1,5
4,4,fake005,"[A1, A2, A3, A4]",1,"[18337, 178, 0]",unknown,west,t3,c2,4,1,1,1,3


In [26]:
data_new.last_time_lapse=data_new.last_time_lapse.map(lambda x:sca(x))
pad_sequence = tf.keras.preprocessing.sequence.pad_sequences
seq_length = 20
time_decay =pad_sequence(data_new.last_time_lapse,maxlen=seq_length,padding='pre',truncating='pre',dtype='float32')
time_decay


In [28]:
time_decay = time_decay.reshape(-1,20,1)
time_decay

array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [1.        ],
        [0.90574783],
        [0.88484824],
        [0.8768634 ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [1.        ],
        [0.00970715],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
      

In [30]:
time_decay.shape

(17, 20, 1)

In [29]:
time_decay_tr = time_decay[tr_idx]
time_decay_te = time_decay[te_idx]

In [32]:
text = data_new.path
    # encoding
t = Tokenizer()
t.fit_on_texts(text)
    # vocabulary size
vocab_size = len(t.word_index) + 1
    # integer encode the documents
encoded_docs = t.texts_to_sequences(text)  
    # padding and truncating path data
newlines =pad_sequence(encoded_docs,maxlen=seq_length,padding='pre',truncating='post')

In [35]:
data_new.path

0     [A1, A2, A3, A4]
1     [A1, A2, A3, A4]
2         [A1, A2, A3]
3     [A1, A2, A3, A4]
4     [A1, A2, A3, A4]
5     [A1, A2, A3, A4]
6         [A1, A2, A3]
7         [A1, A2, A3]
8     [A1, A2, A3, A4]
9     [A1, A2, A3, A4]
10        [A1, A2, A3]
11        [A1, A2, A3]
12    [A1, A2, A3, A4]
13        [A1, A2, A3]
14    [A1, A2, A3, A4]
15        [A1, A2, A3]
16    [A1, A2, A3, A4]
Name: path, dtype: object

In [36]:
encoded_docs

[[1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3, 4],
 [1, 2, 3],
 [1, 2, 3, 4],
 [1, 2, 3],
 [1, 2, 3, 4]]

In [37]:
newlines

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [38]:
X_train = newlines[tr_idx]
Y_train = y[tr_idx]
X_test = newlines[te_idx]
Y_test = y[te_idx]
all_X = np.array(list(map(lambda x: to_categorical(x, num_classes=vocab_size),newlines)), ndmin=3)
X_tr = np.array(list(map(lambda x: to_categorical(x, num_classes=vocab_size), X_train)), ndmin=3)
X_te = np.array(list(map(lambda x: to_categorical(x, num_classes=vocab_size), X_test)), ndmin=3)

In [40]:
X_test

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4]])

In [39]:
X_te

array([[[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        

In [42]:
paths = text[tr_idx].reset_index().path

In [43]:
paths

0         [A1, A2, A3]
1     [A1, A2, A3, A4]
2         [A1, A2, A3]
3     [A1, A2, A3, A4]
4         [A1, A2, A3]
5     [A1, A2, A3, A4]
6     [A1, A2, A3, A4]
7         [A1, A2, A3]
8     [A1, A2, A3, A4]
9     [A1, A2, A3, A4]
10    [A1, A2, A3, A4]
11        [A1, A2, A3]
12        [A1, A2, A3]
13        [A1, A2, A3]
14    [A1, A2, A3, A4]
Name: path, dtype: object

In [44]:
data_lr = data_new.loc[:,['marketing_area','tier','customer_type']]
data_lr

,marketing_area,tier,customer_type
0,north,t1,c1
1,south,t2,c1
2,west,t3,c1
3,east,t3,c2
4,west,t3,c2
5,west,t1,c2
6,west,t2,c1
7,west,t3,c4
8,west,t2,c1
9,south,t1,c2


In [45]:
train_cats(data_lr)

In [51]:
data_lr.dtypes

marketing_area    category
tier              category
customer_type     category
c_type_na            int64
dtype: object

In [46]:
data_lr['c_type'+'_na'] = [1 if v=='NA' else 0 for v in data_lr['customer_type']]

In [47]:
data_lr

,marketing_area,tier,customer_type,c_type_na
0,north,t1,c1,0
1,south,t2,c1,0
2,west,t3,c1,0
3,east,t3,c2,0
4,west,t3,c2,0
5,west,t1,c2,0
6,west,t2,c1,0
7,west,t3,c4,0
8,west,t2,c1,0
9,south,t1,c2,0


In [52]:
for col in data_lr.columns:
    if not is_numeric_dtype(data_lr[col]):
        data_lr[col] = data_lr[col].cat.codes+1

In [53]:
data_lr

,marketing_area,tier,customer_type,c_type_na
0,2,1,1,0
1,3,2,1,0
2,4,3,1,0
3,1,3,2,0
4,4,3,2,0
5,4,1,2,0
6,4,2,1,0
7,4,3,3,0
8,4,2,1,0
9,3,1,2,0


In [54]:
X_tr_lr = data_lr.iloc[tr_idx,:]
X_te_lr = data_lr.iloc[te_idx,:]        
scaler = MinMaxScaler()
scaler.fit(X_tr_lr)
X_tr_lr[['marketing_area', 'tier','customer_type']] = scaler.fit_transform(X_tr_lr[['marketing_area', 'tier','customer_type']])
X_te_lr.loc[:,['marketing_area', 'tier','customer_type']] = scaler.transform(X_te_lr[['marketing_area', 'tier','customer_type']])

In [55]:
X_te_lr

,marketing_area,tier,customer_type,c_type_na
9,0.666667,0.0,0.5,0
14,0.666667,1.0,1.0,0


In [56]:
data_lr.iloc[te_idx,:] 

,marketing_area,tier,customer_type,c_type_na
9,3,1,2,0
14,3,3,3,0


In [57]:
categorical_vars = data_lr.columns[0:3]

In [58]:
categorical_vars

Index(['marketing_area', 'tier', 'customer_type'], dtype='object')